In [1]:
import open3d as o3
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader,random_split
from torchvision import transforms
import torch
import torch.optim as optim
import torch.nn.functional as F

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # Use the first GPU
    print("GPU available.")
else:
    device = torch.device("cpu")
    print("Using CPU.")

GPU available.


## Create Dataloader

In [3]:
def load_data_from_folder(folder_path):
    data = []
    file_list = os.listdir(folder_path)
    for file_name in file_list:
        file_path = os.path.join(folder_path, file_name)
        data.append(file_name)  # Assuming file names are used as data identifiers
    return data

In [4]:
class LiDARDataset(Dataset):
    def __init__(self, clean_folder, noisy_folder):
        self.clean_folder = clean_folder
        self.noisy_folder = noisy_folder
        
        self.clean_data = load_data_from_folder(clean_folder)
        self.noisy_data = load_data_from_folder(noisy_folder)
                
        self.transform = transforms.ToTensor()
    def __len__(self):
        return len(self.clean_data)

    def __getitem__(self, idx):
        
        clean_file_path = os.path.join(self.clean_folder, self.clean_data[idx])
        noisy_file_path = os.path.join(self.noisy_folder, self.noisy_data[idx])
        
        clean_sample = np.load(clean_file_path)['lidar']
        noisy_sample = np.load(noisy_file_path)['lidar']

        clean_sample = self.transform(clean_sample)
        noisy_sample = self.transform(noisy_sample)

        return clean_sample,noisy_sample

In [5]:
dataset = LiDARDataset(clean_folder='data/clean_dataset', noisy_folder='data/noisy_dataset')

In [6]:
def point_cloud_collate(batch):

    original_clouds = [item[0] for item in batch]  # list of original point clouds
    noisy_clouds = [item[1] for item in batch]     # list of noisy point clouds

    return original_clouds, noisy_clouds

In [7]:
# Define sizes for your training and validation sets
train_size = int(0.8 * len(dataset))  # 80% for training
val_size = len(dataset) - train_size  # 20% for validation
# Split the dataset
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
# Create DataLoaders for your train and validation sets
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True,collate_fn=point_cloud_collate)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False,collate_fn=point_cloud_collate)

## Define Model and Hyper Params

In [8]:
import torch
import torch.nn as nn

class DenoisingAutoencoder(nn.Module):
    def __init__(self):
        super(DenoisingAutoencoder, self).__init__()

        # Encoder
        self.encoder =nn.Sequential(
        nn.Conv1d(3, 16, kernel_size=3, stride=2, padding=1),
        nn.ReLU(True),
        nn.Conv1d(16, 32, kernel_size=3, stride=2, padding=1),
        nn.ReLU(True),
        nn.Conv1d(32, 64, kernel_size=3, stride=2, padding=1),
        nn.ReLU(True)
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.ReLU(True),
            nn.ConvTranspose1d(32, 16, kernel_size=4, stride=2, padding=1),
            nn.ReLU(True),
            nn.ConvTranspose1d(16, 3, kernel_size=4, stride=2, padding=1),
            nn.ReLU(True)
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


In [9]:
model = DenoisingAutoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Train Loop

In [14]:
def train(epoch, log_interval=10):
    model.train()
    total_loss = 0
    for batch_idx, batch in enumerate(train_loader):
        # Process each point cloud individually
        original_clouds, noisy_clouds=batch
        noisy_data = []
        clean_data = []
        for point_cloud in noisy_clouds:
            # Ensure point_cloud is a numpy array or a list of numbers
            # If necessary, pad the point cloud to a uniform size here
            point_cloud_tensor = torch.tensor(point_cloud, dtype=torch.float).to(device)
            noisy_data.append(point_cloud_tensor)

        for point_cloud in original_clouds:
            point_cloud_tensor = torch.tensor(point_cloud, dtype=torch.float).to(device)
            clean_data.append(point_cloud_tensor)

        optimizer.zero_grad()
        output = [model(point_cloud.transpose(1, 2)) for point_cloud in noisy_data]

        loss=0
        for o, d in zip(output, clean_data):
            o=o.transpose(1, 2)
            padding_size = d.size(1) - o.size(1)
            padded_tensor_b = F.pad(o, (0, 0, 0, padding_size))
            loss+=criterion(padded_tensor_b, d)

        #loss = sum(criterion(o, d.transpose(1, 2)) for o, d in zip(output, clean_data)) / len(clean_data)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch} Average Training loss: {average_loss:.4f}')

def validate(loader):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_idx, batch in enumerate(loader):
            # Process each point cloud individually
            original_clouds, noisy_clouds=batch
            noisy_data = []
            clean_data = []
            for point_cloud in noisy_clouds:
                # Ensure point_cloud is a numpy array or a list of numbers
                # If necessary, pad the point cloud to a uniform size here
                point_cloud_tensor = torch.tensor(point_cloud, dtype=torch.float).to(device)
                print(point_cloud_tensor.shape)
                noisy_data.append(point_cloud_tensor)
    
            for point_cloud in original_clouds:
                point_cloud_tensor = torch.tensor(point_cloud, dtype=torch.float).to(device)
                clean_data.append(point_cloud_tensor)
    
            output = [model(point_cloud.transpose(1, 2)) for point_cloud in noisy_data]
            loss=0
            for o, d in zip(output, clean_data):
                o=o.transpose(1, 2)
                padding_size = d.size(1) - o.size(1)
                padded_tensor_b = F.pad(o, (0, 0, 0, padding_size))
                loss+=criterion(padded_tensor_b, d)
            val_loss += loss.item()
    val_loss /= len(loader)
    print(f'---------------------------------------------Average Validation loss: {val_loss:.4f}')

In [15]:
epochs = 10
for epoch in range(1, epochs + 1):
    train(epoch)
    validate(val_loader)
torch.save(model.state_dict(), 'denoise_model.pth')

C:\Users\User\AppData\Local\Temp\ipykernel_14676\1448400647.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  point_cloud_tensor = torch.tensor(point_cloud, dtype=torch.float).to(device)
C:\Users\User\AppData\Local\Temp\ipykernel_14676\1448400647.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  point_cloud_tensor = torch.tensor(point_cloud, dtype=torch.float).to(device)
C:\Users\User\AppData\Local\Temp\ipykernel_14676\1448400647.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  point_cloud_tensor = torch.tensor(point_cloud, dtyp

Epoch 1 Average Training loss: 1070.7720
torch.Size([1, 19062, 3])
torch.Size([1, 18173, 3])
---------------------------------------------Average Validation loss: 259.3069
Epoch 2 Average Training loss: 1039.0149
torch.Size([1, 19062, 3])
torch.Size([1, 18173, 3])
---------------------------------------------Average Validation loss: 250.5863
Epoch 3 Average Training loss: 1002.3976
torch.Size([1, 19062, 3])
torch.Size([1, 18173, 3])
---------------------------------------------Average Validation loss: 240.6541
Epoch 4 Average Training loss: 960.7263
torch.Size([1, 19062, 3])
torch.Size([1, 18173, 3])
---------------------------------------------Average Validation loss: 229.4046
Epoch 5 Average Training loss: 913.6312
torch.Size([1, 19062, 3])
torch.Size([1, 18173, 3])
---------------------------------------------Average Validation loss: 217.1801
Epoch 6 Average Training loss: 862.5850
torch.Size([1, 19062, 3])
torch.Size([1, 18173, 3])
---------------------------------------------Avera